In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from sklearn.model_selection import *

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train_input = train.drop(columns="label")
train_label = train["label"]
test_input = test.copy()

In [4]:
train_input

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
size_of_each_image = (int(np.sqrt(train_input.shape[1])),int(np.sqrt(train_input.shape[1])),1)
size_of_each_image

(28, 28, 1)

In [6]:
X_train,X_val,y_train,y_val = train_test_split(train_input,train_label,test_size=0.3,random_state=np.random.randint(0,1000))
X_test = test_input.copy()

In [7]:
X_train_size = tuple([X_train.shape[0]] + list(size_of_each_image))
X_val_size = tuple([X_val.shape[0]] + list(size_of_each_image))
X_test_size = tuple([X_test.shape[0]] + list(size_of_each_image))

In [8]:
X_train_size

(29400, 28, 28, 1)

In [9]:
X_train_1 = np.array(X_train)/255.0
X_val_1 = np.array(X_val)/255.0
X_test_1 = np.array(X_test)/255.0

In [10]:
X_train_2 = X_train_1.reshape(X_train_size)
X_val_2 = X_val_1.reshape(X_val_size)
X_test_2 = X_test_1.reshape(X_test_size)

y_train_2 = np.array(y_train)
y_val_2 = np.array(y_val)

In [11]:
X_train_2

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [12]:
y_train_2

array([5, 4, 9, ..., 4, 4, 5])

In [13]:
test_size = np.unique(y_train).shape[0]
test_size

10

In [14]:
model = tf.keras.models.Sequential(layers=[
    tf.keras.layers.Resizing(height=64,width=64,input_shape=size_of_each_image),
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(filters=128,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128,activation="relu"),
    tf.keras.layers.Dense(units=test_size,activation="softmax")
])

In [15]:
model.compile(
    optimizer = tf.keras.optimizers.legacy.RMSprop(),
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    metrics = tf.keras.metrics.mean_squared_error
)

In [18]:
model.fit(
    x = X_train_2, y = y_train_2,
    batch_size = 32,
    epochs = 100,
    verbose = 1,
    callbacks = tf.keras.callbacks.EarlyStopping(patience=1),
    validation_data = (X_val_2,y_val_2),
    shuffle = True,
    validation_batch_size = 32
)

Epoch 1/100
919/919 [==============================] - 21s 22ms/step - loss: 0.1958 - mean_squared_error: 27.3427 - val_loss: 0.0586 - val_mean_squared_error: 27.5389
Epoch 2/100
919/919 [==============================] - 18s 20ms/step - loss: 0.0504 - mean_squared_error: 27.3500 - val_loss: 0.0452 - val_mean_squared_error: 27.5402
Epoch 3/100
919/919 [==============================] - 18s 19ms/step - loss: 0.0366 - mean_squared_error: 27.3507 - val_loss: 0.0397 - val_mean_squared_error: 27.5409
Epoch 4/100
919/919 [==============================] - 19s 20ms/step - loss: 0.0265 - mean_squared_error: 27.3512 - val_loss: 0.0477 - val_mean_squared_error: 27.5406


In [19]:
cat_cross_entropy_loss, mean_squared_error_metric = model.evaluate(X_test_2)

875/875 [==============================] - 0s 266us/step - loss: 0.0000e+00 - mean_squared_error: 0.0000e+00


In [20]:
cat_cross_entropy_loss

0.0

In [21]:
mean_squared_error_metric

0.0

In [22]:
y_pred = model.predict(X_test_2)

875/875 [==============================] - 5s 5ms/step


In [23]:
y_pred

array([[4.7745502e-10, 9.7030431e-12, 9.9999988e-01, ..., 8.7860563e-08,
        3.3428249e-10, 7.0146872e-10],
       [9.9996948e-01, 5.6868336e-11, 4.7213426e-08, ..., 3.5197846e-08,
        1.4834455e-07, 2.9006118e-05],
       [3.3302519e-09, 2.4003015e-09, 2.4983692e-07, ..., 6.2066835e-07,
        9.9810723e-06, 9.9998093e-01],
       ...,
       [2.3513554e-15, 2.0211928e-11, 9.4135616e-11, ..., 4.5821741e-10,
        2.5690594e-09, 5.3538837e-11],
       [2.0530749e-07, 3.9668935e-09, 6.4452479e-08, ..., 1.7835239e-06,
        2.1378429e-07, 9.9998188e-01],
       [4.0107358e-12, 2.4328387e-12, 1.0000000e+00, ..., 2.3621677e-10,
        8.8880042e-10, 3.8745646e-11]], dtype=float32)

In [24]:
y_test = []
for i in y_pred:
    maxi = np.max(i)
    y_test.append(np.where(i == maxi)[0][0])
y_test = np.array(y_test)

In [25]:
test_df = pd.DataFrame(columns=["ImageId","Label"])
test_df["ImageId"] = range(1,len(y_test)+1)
test_df["Label"] = y_test

In [26]:
test_df.to_csv("amith_submission.csv",index=False)